In [1]:
import napari
from daskoctopus import DaskOctopusLiteLoader

In [2]:
images = DaskOctopusLiteLoader('/home/nathan/data/kraken/test/MK0003/Pos15/Pos15_raw', crop=(1200,1600))

Using cropping: (1200, 1600)


In [3]:
seg = DaskOctopusLiteLoader('/home/nathan/data/kraken/test/MK0003/Pos15/Pos15_stardist_masks')

Using cropping: None


In [4]:
import btrack
from btrack.utils import tracks_to_napari

In [8]:
with btrack.dataio.HDF5FileHandler('/home/nathan/data/kraken/test/MK0003/Pos15/tracking.h5', 'r', obj_type='obj_type_1') as h:
    gfp_tracks = h.tracks

[INFO][2021/11/18 03:11:53 pm] Opening HDF file: /home/nathan/data/kraken/test/MK0003/Pos15/tracking.h5...
[INFO][2021/11/18 03:11:53 pm] Loading tracks/obj_type_1
[INFO][2021/11/18 03:11:54 pm] Loading objects/obj_type_1 (77222, 5) (77222 filtered: None)
[INFO][2021/11/18 03:11:56 pm] Closing HDF file: /home/nathan/data/kraken/test/MK0003/Pos15/tracking.h5


In [9]:
gfp_tracks, properties, graph = tracks_to_napari(gfp_tracks, ndim = 2)

In [10]:
viewer = napari.Viewer()
#viewer.add_image(stack[0], name = "brightfield", contrast_limits=[0,200], blending = 'additive')#, is_pyramid=False)
viewer.add_image(images['gfp'], name="gfp", contrast_limits=[100,255], blending = 'additive', colormap = 'green')#, is_pyramid=False)
viewer.add_image(images['rfp'], name="rfp", contrast_limits=[0,30], blending = 'additive', colormap = 'magenta')#, is_pyramid=False)
viewer.add_image(images['irfp'], name="irfp", contrast_limits=[0,255], blending = 'additive', colormap = 'cyan')#, is_pyramid=False)
viewer.add_image(seg['mask'], name = 'masks')
viewer.add_tracks(gfp_tracks)

<Tracks layer 'gfp_tracks' at 0x7fec4a2da190>